In [ ]:
# Required for extraction of line / word level cropped images from the whole image. Can be done for only pdf images.
! pip install pdfplumber

# Required for converting the pdf images of MTSamples Dataset into images.
! pip install pdf2image

In [ ]:
# As per requirement other imports have also been made alongside the codes where it is used.
import cv2
import PIL
from PIL import Image, ImageSequence
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import yaml
import time
import shutil
import math
import random
import sys
import pickle
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Downloading MTSamples Dataset from Kaggle. Make sure kaggle.json is available in content folder of Colab.

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d shoubhikchakra/mtsamples-dataset
! unzip mtsamples-dataset.zip

Creating the Noisy Dataset from MTSamples dataset

In [ ]:
from skimage.util import random_noise
from sklearn.model_selection import train_test_split

In [ ]:
# Files ending in 0-6 and 8-9 shall be used for training purposes.

! mkdir '/content/noisy_images'

for filename in tqdm(os.listdir('/content/pdf2img/content/pdf2img')):
    if filename.split('.')[0].endswith('0'):
        img = cv2.imread(os.path.join("/content/pdf2img/content/pdf2img", filename), cv2.IMREAD_GRAYSCALE)
        noise_img = random_noise(img, mode='s&p',amount=0.01)
        noise_img = np.array(255*noise_img, dtype = 'uint8')
        cv2.imwrite(os.path.join('/content/noisy_images', filename), noise_img)
    if filename.split('.')[0].endswith('1'):
        img = cv2.imread(os.path.join("/content/pdf2img/content/pdf2img", filename), cv2.IMREAD_GRAYSCALE)
        noise_img = random_noise(img, mode='localvar')
        noise_img = np.array(255*noise_img, dtype = 'uint8')
        cv2.imwrite(os.path.join('/content/noisy_images', filename), noise_img)
    if filename.split('.')[0].endswith('2') or filename.split('.')[0].endswith('8'):
        img = cv2.imread(os.path.join("/content/pdf2img/content/pdf2img", filename), cv2.IMREAD_GRAYSCALE)
        noise_img_1 = random_noise(img, mode='salt', amount = 0.08)
        noise_img_1 = np.array(255*noise_img_1, dtype = 'uint8')
        noise_img_2 = random_noise(noise_img_1, mode='speckle')
        noise_img_2 = np.array(255*noise_img_2, dtype = 'uint8')
        cv2.imwrite(os.path.join('/content/noisy_images', filename), noise_img_2)
    if filename.split('.')[0].endswith('3') or filename.split('.')[0].endswith('9'):
        img = cv2.imread(os.path.join("/content/pdf2img/content/pdf2img", filename), cv2.IMREAD_GRAYSCALE)
        noise_img = random_noise(img, mode='salt',amount=0.1)
        noise_img = np.array(255*noise_img, dtype = 'uint8')
        cv2.imwrite(os.path.join('/content/noisy_images', filename), noise_img)
    if filename.split('.')[0].endswith('4'):
        img = cv2.imread(os.path.join("/content/pdf2img/content/pdf2img", filename), cv2.IMREAD_GRAYSCALE)
        noise_img = random_noise(img, mode='speckle')
        noise_img = np.array(255*noise_img, dtype = 'uint8')
        cv2.imwrite(os.path.join('/content/noisy_images', filename), noise_img)
    if filename.split('.')[0].endswith('5'):
        img = cv2.imread(os.path.join("/content/pdf2img/content/pdf2img", filename), cv2.IMREAD_GRAYSCALE)
        noise_img = random_noise(img, mode='salt',amount=0.09)
        noise_img = np.array(255*noise_img, dtype = 'uint8')
        cv2.imwrite(os.path.join('/content/noisy_images', filename), noise_img)
    if filename.split('.')[0].endswith('6'):
        img = cv2.imread(os.path.join("/content/pdf2img/content/pdf2img", filename), cv2.IMREAD_GRAYSCALE)
        cv2.imwrite(os.path.join('/content/noisy_images', filename), img)

In [ ]:
# Files ending in 7 shall be used for testing purposes.

! mkdir '/content/test_noisy_images'

for filename in tqdm(os.listdir('/content/pdf2img/content/pdf2img')):
    if filename.split('.')[0].endswith('7'):
        img = cv2.imread(os.path.join("/content/pdf2img/content/pdf2img", filename), cv2.IMREAD_GRAYSCALE)
        noise_img = random_noise(img, mode='localvar')
        noise_img = np.array(255*noise_img, dtype = 'uint8')
        cv2.imwrite(os.path.join('/content/test_noisy_images', filename), noise_img)

In [ ]:
datapath = '/content/mtsamples_pdf/mtsamples_pdf'

# Creating Line-Level Dataset

df_line = pd.DataFrame(columns= ['pdf', 'width', 'height', 'linelist'])
for pdfname in tqdm(os.listdir(datapath)):
    with pdfplumber.open(os.path.join(datapath, pdfname), laparams={}) as pdf:
        first_page = pdf.pages[0]
        page = first_page.layout
        lines = []
        for element in page:
            if isinstance(element, LTTextBoxHorizontal):
                for line in element:
                    d = {'x0' : line.x0,
                    'x1' : line.x1,
                    'y0' : line.y0,
                    'y1' : line.y1,
                    'line' : line.get_text()}
                lines.append(d) 
        df_line = df_line.append({'pdf': pdfname, 'width': first_page.width, 'height': first_page.height, 'linelist': lines}, ignore_index=True)


Creating the Training and Testing dataframes and image folders with cropped line-level

In [ ]:
train, test = train_test_split(df_line, test_size=0.1, random_state= 20, shuffle= True)

In [ ]:
train_data = pd.DataFrame(columns=['filename', 'words'])
noisy_data = '/content/noisy_images'
for i, row in tqdm(train.iterrows()):
    count = 0
    filename = row['pdf'].replace('.pdf', '.tiff')
    x_scale, y_scale = (800/row.loc['width']), (1000/row.loc['height'])
    img = cv2.imread(os.path.join(noisy_data, filename))
    try:
        img = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
        for word in row['linelist']:
            text = word["line"]
            encoded = text.encode()
            encoded = str(encoded)
            x1 = math.ceil(word['x0']*x_scale)
            y1 = 1000 - math.ceil(word['y0']*y_scale)
            x2 = math.ceil(word['x1']*x_scale)
            y2 = 1000 - math.ceil(word['y1']*y_scale)
            cropped = img[y2:y1, x1:x2]
            img_name = filename[:-5] + '_' + str(count) + '_cropped.jpg'
            word_name = str(encoded)[2:-3]
            train_data = train_data.append({'filename': img_name, 'words': word_name}, ignore_index=True)
            cv2.imwrite('/content/train_crop/'+img_name, cropped)
            count += 1
    except Exception as e:
        pass

In [ ]:
test_data = pd.DataFrame(columns=['filename', 'words'])
noisy_data = '/content/noisy_images'
for i, row in tqdm(test.iterrows()):
    count = 0
    filename = row['pdf'].replace('.pdf', '.tiff')
    x_scale, y_scale = (800/row.loc['width']), (1000/row.loc['height'])
    img = cv2.imread(os.path.join(noisy_data, filename))
    try:
        img = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
        for word in row['linelist']:
            text = word["line"]
            encoded = text.encode()
            encoded = str(encoded)
            x1 = math.ceil(word['x0']*x_scale)
            y1 = 1000 - math.ceil(word['y0']*y_scale)
            x2 = math.ceil(word['x1']*x_scale)
            y2 = 1000 - math.ceil(word['y1']*y_scale)
            cropped = img[y2:y1, x1:x2]
            img_name = filename[:-5] + '_' + str(count) + '_cropped.jpg'
            word_name = str(encoded)[2:-3]
            test_data = test_data.append({'filename': img_name, 'words': word_name}, ignore_index=True)
            cv2.imwrite('/content/test_crop/'+img_name, cropped)
            count += 1
    except Exception as e:
        pass

In [ ]:
# Confirming that the size of the cropped images folder matches with the size of the respective dataframes.

l = os.listdir('/content/train_crop')
print(len(l))

train_data.drop_duplicates(inplace=True)
print(train_data.shape)

rem = list(set(train_data['filename']) - set(l))
print(len(rem))

for r in rem:
    train_data.drop(train_data[train_data['filename'] == r].index, inplace=True)
    
l = os.listdir('/content/test_crop')
print(len(l))

test_data.drop_duplicates(inplace=True)
print(test_data.shape)

rem = list(set(test_data['filename']) - set(l))
print(len(rem))

for r in rem:
    test_data.drop(test_data[test_data['filename'] == r].index, inplace=True)

Creating the .txt files in the format required for training purposes.

In [ ]:
#converting df into required text file
f = open('/content/rec_train_crop.txt', 'a')
for i in range(len(train_data)):
  f.write('train/{}\t{}\n'.format(train_data.loc[i,'filename'],train_data.loc[i,'words']))  # python will convert \n to os.linesep
f.close()  # you can omit in most cases as the destructor will call it
f = open('/content/rec_test_crop.txt', 'a')
for i in range(len(test_data)):
  f.write('test/{}\t{}\n'.format(test_data.loc[i,'filename'],test_data.loc[i,'words']))  # python will convert \n to os.linesep
f.close()  # you can omit in most cases as the destructor will call it

Training Custom Model

In [ ]:
!pip3 install --upgrade pip
!python3 -m pip install paddlepaddle-gpu==2.0.0 -i https://mirror.baidu.com/pypi/simple #if using gpu
#!python3 -m pip install paddlepaddle-gpu==2.0.0 -i https://mirror.baidu.com/pypi/simple #if non gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 25.0 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://mirror.baidu.com/pypi/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.6/689.6 MB 1.9 MB/s eta 0:00:00
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3


In [ ]:
#cloning the repository
!git clone https://github.com/PaddlePaddle/PaddleOCR

Cloning into 'PaddleOCR'...
remote: Enumerating objects: 38584, done.
remote: Counting objects: 100% (560/560), done.
remote: Compressing objects: 100% (313/313), done.
remote: Total 38584 (delta 324), reused 420 (delta 247), pack-reused 38024
Receiving objects: 100% (38584/38584), 320.53 MiB | 16.75 MiB/s, done.
Resolving deltas: 100% (26834/26834), done.


In [ ]:
#installing the requirements
!pip install importlib-metadata
!pip3 install -r /content/PaddleOCR/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.2/604.2 kB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.5/204.5 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.8/404.8 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 75.1 MB/s eta 0:00:00
  Attempting uninstall: opencv-contrib-python
    Found existing installation: 

In [ ]:
#updated for newer versions
!pip install PyYAML==5.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.6/636.6 kB 35.4 MB/s eta 0:00:00
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
#directory for dataset
#format can be referred in PaddleOCR doc
os.mkdir('/content/PaddleOCR/train_data')

In [ ]:
%cd PaddleOCR/
# Download the pre-trained model of CRNN_mv3_none_bilstm
!wget -P ./pretrain_models/ https://paddleocr.bj.bcebos.com/dygraph_v2.0/en/rec_mv3_none_bilstm_ctc_v2.0_train.tar
%cd pretrain_models
!tar -xf rec_mv3_none_bilstm_ctc_v2.0_train.tar && rm -rf rec_mv3_none_bilstm_ctc_v2.0_train.tar

/content/PaddleOCR
--2022-07-04 17:53:46--  https://paddleocr.bj.bcebos.com/dygraph_v2.0/en/rec_mv3_none_bilstm_ctc_v2.0_train.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.46.61, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51200000 (49M) [application/x-tar]
Saving to: ‘./pretrain_models/rec_mv3_none_bilstm_ctc_v2.0_train.tar’

rec_mv3_none_bilstm 100%[===================>]  48.83M  25.3MB/s    in 1.9s    

2022-07-04 17:53:48 (25.3 MB/s) - ‘./pretrain_models/rec_mv3_none_bilstm_ctc_v2.0_train.tar’ saved [51200000/51200000]

/content/PaddleOCR/pretrain_models


After making necessary changes in the configuration file i.e, dataset directory, training parameters etc.

Training the model with pretrained weights and if want to train without pretrained weights, ignore from '-o'

In [ ]:
#training the model
%cd /content/PaddleOCR
#!python3 tools/train.py -c configs/rec/rec_mv3_none_bilstm_ctc.yml
!python3 tools/train.py -c configs/rec/rec_mv3_none_bilstm_ctc.yml -o Global.pretrained_model=/content/rec/mv3_none_bilstm_ctc/best_accuracy.pdparams

/content/PaddleOCR
/usr/local/lib/python3.7/dist-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the b

In [ ]:
#infering the trained model without exporting
%cd PaddleOCR
!python3 tools/infer_rec.py -c configs/rec/rec_mv3_none_bilstm_ctc.yml -o Global.pretrained_model=/content/PaddleOCR/output/rec/mv3_none_bilstm_ctc/best_accuracy.pdparams Global.infer_img=/content/PaddleOCR/doc/imgs_words/en/word_1.png

/content/PaddleOCR
/usr/local/lib/python3.7/dist-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the b

In [ ]:
!python3 tools/infer_rec.py -c configs/rec/rec_mv3_none_bilstm_ctc.yml -o Global.pretrained_model=/content/PaddleOCR/output/rec/mv3_none_bilstm_ctc/best_accuracy.pdparams Global.infer_img=/content/PaddleOCR/train_data/content/test_crop/1570_2_cropped.jpg

/usr/local/lib/python3.7/dist-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To si

In [ ]:
%cd /content/PaddleOCR
!python3 tools/infer_rec.py -c configs/rec/rec_mv3_none_bilstm_ctc.yml -o Global.pretrained_model=/content/PaddleOCR/output/rec/mv3_none_bilstm_ctc/best_accuracy.pdparams Global.infer_img=/content/PaddleOCR/train_data/content/test_crop/1570_20_cropped.jpg

/content/PaddleOCR
/usr/local/lib/python3.7/dist-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the b

In [ ]:
!python3 tools/infer_rec.py -c configs/rec/rec_mv3_none_bilstm_ctc.yml -o Global.pretrained_model=/content/PaddleOCR/output/rec/mv3_none_bilstm_ctc/best_accuracy.pdparams Global.infer_img=/content/PaddleOCR/train_data/content/test_crop/1570_0_cropped.jpg

/usr/local/lib/python3.7/dist-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To si

In [ ]:
#Exporting the model for inferencing
!python3 tools/export_model.py -c configs/rec/rec_mv3_none_bilstm_ctc.yml -o Global.pretrained_model=/content/PaddleOCR/output/rec/mv3_none_bilstm_ctc/best_accuracy  Global.save_inference_dir=./inference/CRNN_mv3/


/usr/local/lib/python3.7/dist-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To si

In [ ]:
#Prediction with the inferenced model
!python3 tools/infer/predict_rec.py --image_dir="/content/PaddleOCR/train_data/content/test_crop/1570_0_cropped.jpg" --rec_model_dir="/content/PaddleOCR/inference/CRNN_mv3" --rec_image_shape="3, 32, 320" --rec_char_dict_path="/content/PaddleOCR/ppocr/utils/dict/en_dict.txt" --use_gpu=False

In [ ]:
# Download DB text detection inference model for concatenating it with text recogniser
# Here directly inferenc model is downloaded for use
# We can also download pretrained models and train and export them for inferencing
!wget  https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_slim_infer.tar
!tar xf en_PP-OCRv3_det_slim_infer.tar

--2022-07-08 05:21:09--  https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_slim_infer.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.46.61, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2726400 (2.6M) [application/x-tar]
Saving to: ‘en_PP-OCRv3_det_slim_infer.tar’

en_PP-OCRv3_det_sli 100%[===================>]   2.60M   734KB/s    in 4.3s    

2022-07-08 05:21:14 (624 KB/s) - ‘en_PP-OCRv3_det_slim_infer.tar’ saved [2726400/2726400]



In [ ]:
#testing with sample image
from PIL import Image
im = Image.open(r'/content/drive/MyDrive/1.tiff')
im.save('test.jpeg')

In [ ]:
#Prediction with inferenced model
!python3 tools/infer/predict_rec.py --image_dir="./doc/imgs_words/en/word_1.png" --rec_model_dir="./inference/CRNN_mv3/" --rec_image_shape="3, 32, 320" --rec_char_dict_path="./ppocr/utils/en_dict.txt" --use_gpu=False


/usr/local/lib/python3.7/dist-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To si

In [ ]:
#Concatenating both detection and recognition
#mention required parameters like dict, gpu , image_shape etc if needed
#detection + recognition
!python3 tools/infer/predict_system.py --image_dir="./test.jpeg" --det_model_dir="./en_PP-OCRv3_det_slim_infer" --rec_model_dir="./inference/CRNN_mv3" --rec_image_shape="3, 32, 320" --use_angle_cls=false --rec_char_dict_path="./ppocr/utils/en_dict.txt" --use_gpu=False

/usr/local/lib/python3.7/dist-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To si

#Every inference is taken wrt cpu.
##Best predicted time observed was around 4-5 seconds for the detection+recognition.